In [113]:
import geopandas
rhodes = geopandas.read_file('rhodes.gpkg')
bbox = rhodes.total_bounds

In [114]:
import pystac_client

api_url = "https://earth-search.aws.element84.com/v1"
collection_id = "sentinel-2-c1-l2a"

client = pystac_client.Client.open(api_url)
search = client.search(
    collections=[collection_id],
    datetime="2023-07-01/2023-08-31",
    bbox=bbox
)

item_collection = search.item_collection()

In [115]:
import odc.stac
ds = odc.stac.load(
    item_collection,
    groupby='solar_day',
    chunks={'x': 2048, 'y': 2048},
    use_overviews=True,
    resolution=20,
    bbox=rhodes.total_bounds,
)

In [116]:
green = ds['green']
blue = ds['blue']
red = ds['red']
nir = ds['nir']

swir16 = ds['swir16']
swir22 = ds['swir22']



scl = ds['scl']

In [117]:
# generate mask ("True" for pixel being cloud or water)
mask = scl.isin([
    #3,  # CLOUD_SHADOWS
    6,  # WATER
    #8,  # CLOUD_MEDIUM_PROBABILITY
    #9,  # CLOUD_HIGH_PROBABILITY
    10  # THIN_CIRRUS
])
green_masked = green.where(~mask)
blue_masked = blue.where(~mask)
red_masked = red.where(~mask)
nir_masked = nir.where(~mask)

swir16_masked = swir16.where(~mask)
swir22_masked = swir22.where(~mask)


In [109]:
import rioxarray

In [110]:
ndvi = (nir_masked - red_masked) / (nir_masked + red_masked)
ndwi = (green_masked - nir_masked)/(green_masked + nir_masked)
index = (swir16_masked - swir22_masked)/(swir16_masked + swir22_masked)


In [ ]:
ndvi.rio.resolution(), ndwi.rio.resolution(), index.rio.resolution()

In [ ]:
index_match = index.rio.reproject_match(ndvi)
swir16_masked_match = swir16_masked.rio.reproject_match(ndvi)

In [77]:
burned = (
    (ndvi <= 0.3) & 
    (ndwi <= 0.1) &
    ((index_match + nir_masked/10_000) <= 0.1) &
    ((blue_masked/10_000) <= 0.1) & 
    ((swir16_masked_match/10_000) >= 0.1)
)

In [91]:
burned_after = burned.sel(time="2023-08-27")


In [92]:
burned_int_after = burned_after.astype(int)

In [81]:
import rioxarray  # Enables geospatial operations with xarray
import rasterio
from rasterio.transform import from_bounds

def export_to_geotiff(data_array, output_path):
    """
    Export an xarray.DataArray to a GeoTIFF file.
    
    Parameters:
    - data_array (xarray.DataArray): The data to be exported.
    - output_path (str): The file path where the GeoTIFF will be saved.
    """
    # Ensure the DataArray has the proper CRS
    if not data_array.rio.crs:
        raise ValueError("DataArray must have a CRS. Use .rio.write_crs() to assign one.")
    
    # Squeeze the DataArray to remove any extra dimensions
    data_2d = data_array.squeeze()

    # Verify that the data is now 2-dimensional
    if data_2d.ndim != 2:
        raise ValueError("The data array must be 2-dimensional after squeezing.")

    # Get the transform and CRS from the data
    transform = from_bounds(
        *data_2d.rio.bounds(),
        data_2d.shape[1],  # width (x)
        data_2d.shape[0]   # height (y)
    )

    # Export the DataArray to a GeoTIFF
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=data_2d.shape[0],
        width=data_2d.shape[1],
        count=1,  # Number of bands (single band for this function)
        dtype=data_2d.dtype,
        crs=data_2d.rio.crs,  # CRS from the data
        transform=transform,  # Affine transform
    ) as dst:
        dst.write(data_2d.values, 1)
        
    print(f"GeoTIFF exported successfully to {output_path}")

In [93]:
burned_int_after = burned_int_after.squeeze()

In [94]:
burned_int_after = burned_int_after.rio.write_crs("EPSG:32635")

In [ ]:
export_to_geotiff(burned_int_after, "burned_after4.tif")

In [118]:
import rioxarray  # Enables geospatial operations with xarray
import rasterio
from rasterio.transform import from_bounds

def export_rgb_to_geotiff(red, green, blue, output_path):
    """
    Export red, green, and blue xarray.DataArrays as a true color RGB GeoTIFF.
    
    Parameters:
    - red (xarray.DataArray): The red band data.
    - green (xarray.DataArray): The green band data.
    - blue (xarray.DataArray): The blue band data.
    - output_path (str): The file path where the GeoTIFF will be saved.
    """
    # Ensure each DataArray has the proper CRS and squeeze extra dimensions
    if not red.rio.crs or not green.rio.crs or not blue.rio.crs:
        raise ValueError("Each DataArray must have a CRS. Use .rio.write_crs() to assign one.")
    
    # Ensure they all share the same CRS and bounds
    if not (red.rio.crs == green.rio.crs == blue.rio.crs):
        raise ValueError("All input bands must have the same CRS.")
    
    # Squeeze the DataArrays to remove any extra dimensions
    red_2d = red.squeeze()
    green_2d = green.squeeze()
    blue_2d = blue.squeeze()

    # Ensure that the dimensions match
    if red_2d.shape != green_2d.shape or green_2d.shape != blue_2d.shape:
        raise ValueError("All input bands must have the same shape.")
    
    # Get the transform and CRS from one of the bands
    transform = from_bounds(
        *red_2d.rio.bounds(),
        red_2d.shape[1],  # width (x)
        red_2d.shape[0]   # height (y)
    )
    crs = red_2d.rio.crs

    # Stack the RGB data into a 3D array (bands, height, width)
    rgb_data = [red_2d.values, green_2d.values, blue_2d.values]
    
    # Export the RGB data to a GeoTIFF
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=red_2d.shape[0],
        width=red_2d.shape[1],
        count=3,  # Three bands for RGB
        dtype=red_2d.dtype,  # Assume all bands have the same dtype
        crs=crs,  # CRS from the data
        transform=transform,  # Affine transform
    ) as dst:
        dst.write(rgb_data[0], 1)  # Write red band
        dst.write(rgb_data[1], 2)  # Write green band
        dst.write(rgb_data[2], 3)  # Write blue band
    
    print(f"RGB GeoTIFF exported successfully to {output_path}")

# Example usage:
export_rgb_to_geotiff(red_masked, green_masked, blue_masked, "true_color_image_4.tif")


ValueError: Source shape (1, 25, 3255, 2567) is inconsistent with given indexes 1